# MOUSENEU_LP_N2V_1: Noise2Void 2D on Mouse Neuriter Dataset

Dataset is composed one video of mouse neurites for training and one for validation.
Videos have size (6500, 490, 490) and are acquired using 66mW power, 30hz frame rate and 40um depth.

In [ ]:
#!pip install microssim

In [1]:
import tifffile
from pathlib import Path
from microssim import MicroSSIM, micro_structural_similarity
from skimage.metrics import structural_similarity
from careamics.utils.metrics import psnr, scale_invariant_psnr
import numpy as np
import pandas as pd

DATASET_PATH = Path("/localscratch/calcium_imaging_dataset/calcium_imaging/MOUSENEU_LP/")
PREDICTIONS_PATH = Path("output")
MODELS_TO_COMPARE = ['MOUSENEU_LP_N2N_1', 'MOUSENEU_LP_N2V_1', 'MOUSENEU_LP_DEEPCAD_1']
SPLIT_TO_COMPARE = ['train', 'val']

In [ ]:
results = []


for split in SPLIT_TO_COMPARE:
    input_paths = list(DATASET_PATH.joinpath(f"{split}").glob("*.tif"))
    for input_path in input_paths:
        filename = str(input_path.name)
        gt_path = DATASET_PATH.joinpath(f"{split}_gt").joinpath(filename.replace("lowSNR", "highSNR"))
        
        gt_image = tifffile.imread(gt_path).astype(np.uint16)
        predictions_paths = [(PREDICTIONS_PATH.joinpath(model).joinpath(f"{split}").joinpath(filename), model) for model in MODELS_TO_COMPARE]

        for target_path, target_name in [(input_path, "lowSNR")] + predictions_paths:
            target_image = tifffile.imread(target_path).astype(np.uint16)
            ms = MicroSSIM()
            # Fit microSSIM parameters with 1/10 of the frame to avoid OOM
            ms.fit(gt=gt_image[::10], pred=target_image[::10])

            for f, (gt_frame, target_frame) in enumerate(zip(gt_image, target_image)):
                res = dict()
                res["frame"] = f
                res["target_name"] = target_name
                res["split"] = split
                res["gt_fp"] = gt_path.name
                res["target_fp"] = target_path.name
                res["ssim"] = structural_similarity(gt_frame, target_frame, data_range=65535)
                res["microssim"] = ms.score(gt=gt_frame, pred=target_frame)
                res["psnr"] = psnr(gt=gt_frame, pred=target_frame, data_range=65535)
                res["si_psnr"] = scale_invariant_psnr(gt=gt_frame, pred=target_frame)
                results.append(res)


In [ ]:
pd_results = pd.DataFrame.from_dict(results)
pd_results.to_csv("results/MOUSENEU_LP.csv")

In [ ]:
means = pd_results.groupby(["split","target_name", "gt_fp", "target_fp"]).mean().drop(columns=["frame"])
std = pd_results.groupby(["split","target_name", "gt_fp", "target_fp"]).std().drop(columns=["frame"])
formatted_df = means.copy()
for col in means.columns:
    formatted_df[col] = means[col].map('{:.3f}'.format) + " ± " + std[col].map('{:.3f}'.format)



In [ ]:
formatted_df.to_csv("results/MOUSENEU_LP_formatted.csv")

In [ ]:
import pandas as pd
formatted_df = pd.read_csv("results/MOUSENEU_LP_formatted.csv", index_col=0)
with open("results/MOUSENEU_LP.md", "w") as f:
    f.write(formatted_df.to_markdown())

# Generate Previews

In [ ]:
import tifffile
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from view_utils import create_gif_from_numpy


EXPERIMENT_NAMES = ["MOUSENEU_LP_N2N_1", "MOUSENEU_LP_N2V_1", "MOUSENEU_LP_DEEPCAD_1", "MOUSENEU_LP_CELLMINCER_1"]

for split in ['train', 'val']:
    input_tiff_path = list(DATASET_PATH.joinpath(split).rglob("*.tif"))[0]
    input_name = input_tiff_path.name
    base_folder = Path(f"docs/experiments_preview/MOUSENEU_LP/{split}/")
    base_folder.mkdir(parents=True, exist_ok=True)
    print(f"Processing {input_name}")

    input_image = tifffile.imread(input_tiff_path)        
    create_gif_from_numpy(input_image, str(base_folder.joinpath("input.gif")), scale_factor=0.5, frame_skip=20)

    for exp_name in EXPERIMENT_NAMES:
        print(f"Processing {exp_name}")
        if "CELLMINCER" in exp_name:
            output_image = tifffile.imread(str(Path("output").joinpath(exp_name).joinpath(split).joinpath("denoised_tyx.tif")))
        else: 
            output_image = tifffile.imread(str(Path("output").joinpath(exp_name).joinpath(split).joinpath(input_name)))

        create_gif_from_numpy(output_image, str(base_folder.joinpath(f"{exp_name}.gif")), scale_factor=0.5, frame_skip=20)
                    


Processing 01_MouseNeurite_GCaMP6f_66mWpower_40umdepth_30Hz_lowSNR_MCRound1.tif


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [3]:
ac = auto_contrast(input_image)
ac

array([[[19920, 19856, 20027, ..., 19849, 20172, 20235],
        [19946, 19976, 20537, ..., 20094, 20407, 19753],
        [19912, 19894, 19872, ..., 20193, 19813, 21167],
        ...,
        [19891, 19884, 19984, ..., 20634, 18981, 20421],
        [19552, 20088, 20024, ..., 19902, 20103, 19860],
        [19905, 20041, 19977, ..., 19887, 20814, 19691]],

       [[19863, 19952, 19899, ..., 20001, 19844, 20004],
        [19930, 19927, 19894, ..., 19835, 19753, 19837],
        [20013, 19829, 19934, ..., 20684, 21583, 21056],
        ...,
        [19880, 19964, 19847, ..., 19961, 19849, 19956],
        [19943, 19915, 20036, ..., 19933, 19906, 19900],
        [19922, 19837, 19970, ..., 19945, 19880, 19905]],

       [[20631, 20253, 19936, ..., 20380, 19676, 20340],
        [20714, 19871, 19883, ..., 19884, 19853, 19695],
        [19831, 19872, 19862, ..., 20299, 20321, 20383],
        ...,
        [19868, 19915, 19917, ..., 19958, 19890, 19887],
        [20055, 19915, 19943, ..., 19652, 199

In [4]:
ac.shape

(6500, 490, 490)